In [47]:
from __future__ import division

label='returnQuantity'

import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
from scipy import stats
from sklearn.cross_validation import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder

In [48]:
def dmc_error(label, prediction):
    #∑ |returnQuantity𝑖 − prediction𝑖 |
    return np.sum(np.abs( np.subtract(label, prediction) ))

def feature_importances(Xdf, rfc):
    pair=zip(Xdf.columns, rfc.feature_importances_)
    t1=[i for i in pair]; t1
    return sorted(t1, key=itemgetter(1))

def rbf_svm(X_train, y_train, X_test, C):
    from sklearn.svm import SVC
    clf = SVC( C=C, kernel='rbf')
    clf.fit(X_train,y_train)
    return clf.predict(X_test)

def randfor(X_train, y_train, X_test, est):
    from sklearn.ensemble import RandomForestClassifier
    clf = RandomForestClassifier(n_estimators = est, criterion = 'gini', random_state = 1234)
    clf.fit(X_train, y_train)
    return clf.predict(X_test)

In [49]:
from datetime import datetime
df = pd.read_pickle('train_gue2.pkl')

In [50]:
object_columns=df.loc[:, df.dtypes == object].columns
df= df.drop(object_columns, axis=1)

In [51]:
Xdf=df.drop([label,'months','colorCode','productGroup', 'deviceID','rrp','price','voucherAmount','after_voucher'], axis=1)
X=Xdf.values
y=df.returnQuantity.values

In [52]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)

In [53]:
print datetime.now()

lr=LinearRegression(n_jobs=-1, normalize=True)
lr.fit(X_train, y_train)
y_pred=lr.predict(X_test)
y_pred=np.array([round(yp) for yp in y_pred], dtype=np.int8)
y_pred[y_pred<0]=0
print confusion_matrix(y_test,y_pred)
print dmc_error(y_test,y_pred)
    
print datetime.now()

2016-05-11 20:22:24.300247
[[167672      0      0      0      0      0]
 [     0 180524      0      0      0      0]
 [     0    545      0      0      0      0]
 [     0     26      0      0      0      0]
 [     0      3      0      0      0      0]
 [     0      5      0      0      0      0]]
626
2016-05-11 20:23:26.691733


In [28]:
def time_series_validation(n, step):
    """Cross validation for time-series data."""
    leap = 100; k = 2200; h = 650
    indices = np.arange(n)
    for i in range(step):
        yield indices[:k], indices[k:k+h]
        k += leap

In [29]:
def validation(X,y):
    """Local Cross Validation.
    """
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)
    regress = LinearRegression(n_jobs=-1, normalize=True)

    print(" --- Start local evaluation.")
    # kf = cross_validation.KFold(X_coupons.shape[0], n_folds=10)
    # ss = cross_validation.ShuffleSplit(X_coupons.shape[0], n_iter=100, train_size=0.9)
    tsv = time_series_validation(X_train.shape[0], 33)
    i=1; scores = []
    for train, test in tsv:
        size = test.shape[0]

        regress.fit(X_train[train], y_train[train])
        y_preds = np.exp(regress.predict(X_train[test]))
        y_preds = y_preds.reshape(size, 1)

        y_true = y_train[test].reshape(X_train[test].shape[0], 1)
        
        score = dmc_error(y_true, y_preds)

        print(" ---- Score of #{0} : {1:.5f}".format(i, score))
        i += 1
        scores.append(score)

    print(" --- Finished local evaluation.")
    quantiles = stats.mstats.mquantiles(scores)
    print(" --- Score Results:")
    print(" - min: {:.5f}".format(np.min(scores)))
    print(" - 25%: {:.5f}".format(quantiles[0]))
    print(" - median: {:.5f}".format(np.median(scores)))
    print(" - 75%: {:.5f}".format(quantiles[2]))
    print(" - max: {:.5f}".format(np.max(scores)))
    print(" - mean: {0:.5f} (+/-{1:.5f})".format(np.mean(scores), stats.sem(scores)))

In [30]:
validation(X,y)

 --- Start local evaluation.
 ---- Score of #1 : 893.00939
 ---- Score of #2 : 896.85903
 ---- Score of #3 : 897.05833
 ---- Score of #4 : 904.26207
 ---- Score of #5 : 900.71143
 ---- Score of #6 : 898.54793
 ---- Score of #7 : 896.35610
 ---- Score of #8 : 894.51887
 ---- Score of #9 : 891.65619
 ---- Score of #10 : 897.30749
 ---- Score of #11 : 898.84775
 ---- Score of #12 : 906.08629
 ---- Score of #13 : 909.47333
 ---- Score of #14 : 908.29445
 ---- Score of #15 : 911.22435
 ---- Score of #16 : 905.37379
 ---- Score of #17 : 899.68837
 ---- Score of #18 : 894.01575
 ---- Score of #19 : 883.68090
 ---- Score of #20 : 885.73610
 ---- Score of #21 : 891.10522
 ---- Score of #22 : 887.39037
 ---- Score of #23 : 890.18540
 ---- Score of #24 : 889.50348
 ---- Score of #25 : 898.00935
 ---- Score of #26 : 899.75675
 ---- Score of #27 : 899.00863
 ---- Score of #28 : 899.39555
 ---- Score of #29 : 901.53997
 ---- Score of #30 : 899.81793
 ---- Score of #31 : 900.49600
 ---- Score of #32 

In [57]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2325165 entries, 0 to 2325164
Data columns (total 27 columns):
colorCode             int32
productGroup          float16
quantity              int32
price                 float16
rrp                   float16
voucherAmount         float16
deviceID              int32
returnQuantity        int32
binaryLabel           int32
months                int8
seasons               int32
order_total           float32
customer_budget       float32
expense_ratio         float16
unit_price            float32
usual_unit_price      float32
price_diff            float32
after_voucher         float16
order_order           int32
choice_order          int32
articleID_cumprob     float16
colorCode_cumprob     float16
customerID_cumprob    float16
sizeCode_cumprob      float16
ac_cumprob            float64
as_cumprob            float64
proSea_cumprob        float64
dtypes: float16(10), float32(5), float64(3), int32(8), int8(1)
memory usage: 232.8 MB
